# New Age

Fakulteta za matematiko in fiziko - Univerza v Ljubljani

Ivan Slapničar, Sveučilište u Splitu, FESB

April, 2017

## Messages

Easy -> hard?

Present: __age of search__ -> _mathematics_

Future: __age of recommendation__  -> _mathematics_

(BigData, new technologies)


## Easy -> hard

weight:  1 _kg_  -> 1 _kg_ $\pm$ 0.000000001 _kg_

running: 100 _m_ -> 42,195 _m_ ili 100 _m_ < 10 _sek_

mathematics: exam -> state competition  -> [Olympiad](http://www.imo-official.org/)

search, recommending: good -> excellent

## Age of Search

google (and others)


* [50 billion pages](http://www.worldwidewebsize.com/), [3.5 billion querries daily](http://www.internetlivestats.com/google-search-statistics/)
* __PageRank__
* history, context - cookies, storing data (about you), [200+ parameters](http://backlinko.com/google-ranking-factors)

## PageRank

* Graph theory and linear algebra
* [C, Moler, Google PageRank][Mol11]


[Mol11]: https://www.mathworks.com/moler/exm/chapters/pagerank.pdf "C, Moler, 'Google PageRank', mathWorks, 2011."


In [1]:
i = vec([ 2 6 3 4 4 5 6 1 1])
j = vec([ 1 1 2 2 3 3 3 4 6])
G=sparse(i,j,1.0)

6×6 SparseMatrixCSC{Float64,Int64} with 9 stored entries:
  [2, 1]  =  1.0
  [6, 1]  =  1.0
  [3, 2]  =  1.0
  [4, 2]  =  1.0
  [4, 3]  =  1.0
  [5, 3]  =  1.0
  [6, 3]  =  1.0
  [1, 4]  =  1.0
  [1, 6]  =  1.0

In [2]:
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  1.0  0.0  0.0  0.0

On the graph we do a _random walk_ - probabilty to follow any link is equal:

In [3]:
c=sum(G,1)
n=size(G,1)
for j=1:n
    if c[j]>0
        G[:,j]=G[:,j]/c[j]
    end
end
full(G)

6×6 Array{Float64,2}:
 0.0  0.0  0.0       1.0  0.0  1.0
 0.5  0.0  0.0       0.0  0.0  0.0
 0.0  0.5  0.0       0.0  0.0  0.0
 0.0  0.5  0.333333  0.0  0.0  0.0
 0.0  0.0  0.333333  0.0  0.0  0.0
 0.5  0.0  0.333333  0.0  0.0  0.0

* $p$: probability to follow link
* $1-p$: probability to visit some other page (at random)
* google uses $p=0.85$ ?

In [4]:
p=0.85
z = ((1-p)*(c.!=0) + (c.==0))/n
A=p*G+ones(n)*z

6×6 Array{Float64,2}:
 0.025  0.025  0.025     0.875  0.166667  0.875
 0.45   0.025  0.025     0.025  0.166667  0.025
 0.025  0.45   0.025     0.025  0.166667  0.025
 0.025  0.45   0.308333  0.025  0.166667  0.025
 0.025  0.025  0.308333  0.025  0.166667  0.025
 0.45   0.025  0.308333  0.025  0.166667  0.025

## Idea

Let us start a random walk from the vector $x_0=\begin{bmatrix} 1/n \\ 1/n \\ \vdots \\ 1/n \end{bmatrix}$.

The subsequent vectors are  

$$
x_1=A\cdot x_0 \\
x_2=A\cdot x_1 \\
x_3=A\cdot x_2\\
\vdots
$$

When the vector __stabilizes__:

$$
A\cdot x\approx x,
$$

then $x[i]$ is the __rank of the page__ $i$.

In [5]:
function myPageRank(G::SparseMatrixCSC{Float64,Int64},steps::Int)
    p=0.85
    c=sum(G,1)/p
    n=size(G,1)
    for i=1:n
        G.nzval[G.colptr[i]:G.colptr[i+1]-1]./=c[i]
    end
    e=ones(n)
    x=e/n
    z = vec(((1-p)*(c.!=0) + (c.==0))/n)
    for j=1:steps
        x=G*x+(z⋅x)
    end
    x/norm(x,1)
end

myPageRank (generic function with 1 method)

In [6]:
myPageRank(G,15)

6-element Array{Float64,1}:
 0.321024 
 0.170538 
 0.106596 
 0.136795 
 0.0643103
 0.200737 

In [7]:
fieldnames(G)

5-element Array{Symbol,1}:
 :m     
 :n     
 :colptr
 :rowval
 :nzval 

In [8]:
G

6×6 SparseMatrixCSC{Float64,Int64} with 9 stored entries:
  [2, 1]  =  0.425
  [6, 1]  =  0.425
  [3, 2]  =  0.425
  [4, 2]  =  0.425
  [4, 3]  =  0.283333
  [5, 3]  =  0.283333
  [6, 3]  =  0.283333
  [1, 4]  =  0.85
  [1, 6]  =  0.85

In [9]:
G.colptr

7-element Array{Int64,1}:
  1
  3
  5
  8
  9
  9
 10

### [Stanford web graph](http://snap.stanford.edu/data/web-Stanford.html)

This is somewhat larger test problem.

In [10]:
W=readdlm("web-Stanford.txt",Int)

2312497×2 Array{Int64,2}:
      1    6548
      1   15409
   6548   57031
  15409   13102
      2   17794
      2   25202
      2   53625
      2   54582
      2   64930
      2   73764
      2   84477
      2   98628
      2  100193
      ⋮        
 281849  165189
 281849  177014
 281849  226290
 281849  243180
 281849  244195
 281849  247252
 281849  281568
 281865  186750
 281865  225872
 281888  114388
 281888  192969
 281888  233184

In [11]:
S=sparse(W[:,2],W[:,1],1.0)

281903×281903 SparseMatrixCSC{Float64,Int64} with 2312497 stored entries:
  [6548  ,      1]  =  1.0
  [15409 ,      1]  =  1.0
  [17794 ,      2]  =  1.0
  [25202 ,      2]  =  1.0
  [53625 ,      2]  =  1.0
  [54582 ,      2]  =  1.0
  [64930 ,      2]  =  1.0
  [73764 ,      2]  =  1.0
  [84477 ,      2]  =  1.0
  [98628 ,      2]  =  1.0
  ⋮
  [168703, 281902]  =  1.0
  [180771, 281902]  =  1.0
  [266504, 281902]  =  1.0
  [275189, 281902]  =  1.0
  [44103 , 281903]  =  1.0
  [56088 , 281903]  =  1.0
  [90591 , 281903]  =  1.0
  [94440 , 281903]  =  1.0
  [216688, 281903]  =  1.0
  [256539, 281903]  =  1.0
  [260899, 281903]  =  1.0

In [12]:
@time x100=myPageRank(S,100);

  4.834073 seconds (282.42 k allocations: 488.428 MiB, 7.51% gc time)


In [13]:
x101=myPageRank(S,101)
maximum(abs,(x100-x101)./x100)

2.3491380779056936e-7

In [14]:
sortperm(x100,rev=true), sort(x100,rev=true)

([89073, 226411, 241454, 262860, 134832, 234704, 136821, 68889, 105607, 69358  …  281647, 281700, 281715, 281728, 281778, 281785, 281813, 281849, 281865, 281888], [0.0113029, 0.00926783, 0.00829727, 0.00302312, 0.00300128, 0.00257173, 0.00245371, 0.00243079, 0.00239105, 0.00236401  …  5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7, 5.33369e-7])

## Age of Recommendation

NetFlix, Amazon Prime, PickBox, ... - on-line streaming of movies and shows

[NetFlix](https://www.netflix.com/hr/)

 * [80 million users](https://www.statista.com/statistics/250934/quarterly-number-of-netflix-streaming-subscribers-worldwide/), 5,000 movies
 * [NetFlix Prize](http://www.kdd.org/kdd2014/tutorials/KDD%20-%20The%20Recommender%20Problem%20Revisited.pdf)
 

## Mathematics

Netflix Recommendation Engine is based on approximation of a (large and sparse) matrix
```
M = Users x Movies 
```
using (approximation of) singular value decomposition (SVD): 

* [IncrementalSVD.jl](https://github.com/aaw/IncrementalSVD.jl)
* [A parallel recommendation engine in Julia](http://juliacomputing.com/blog/2016/04/22/a-parallel-recommendation-engine-in-julia.html)

## Similarities

Similarity of users $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[i,:],M[k,:])}{\|M[i,:]\| \cdot \|M[k,:]\|}
$$
Similarity of movies $i$ and $k$:
$$
\cos \angle (i,k)=\frac{(M[:,i],M[:,k)}{\|M[:,i]\| \cdot \|M[:,k]\|}
$$

## Search

Row $M[u,:]$ - what user $u$ thinks about movies

Column $M[:,m]$ - what users think about movie $m$

Element $M[u,m]$ - what user $u$ thinks about movie $m$.

## Problem

Matrix $M$ is sparse so we do not have enough information. For example, 

```
900188 marks / ( 6040 users x 3706 movies ) = 4%
```

## Approximation

SVD decomposition $M=U\Sigma V^T$ is [approximated by a low-rank matrix](https://en.wikipedia.org/wiki/Low-rank_approximation) (for example. $rank=25$)

![SVD decomposition](svd.png)

The approximation matrix is __full__ and __gives enough good information__.

Prize for efficient approximation algorithm was $\$$ 1.000.000.  

In [15]:
# Pkg.clone("git://github.com/aaw/IncrementalSVD.jl.git")

In [16]:
using IncrementalSVD

In [17]:
whos(IncrementalSVD)

                IncrementalSVD     52 KB     Module
                        Rating    136 bytes  DataType
                     RatingSet    148 bytes  DataType
                  RatingsModel    160 bytes  DataType
             cosine_similarity      0 bytes  IncrementalSVD.#cosine_similarity
          get_predicted_rating      0 bytes  IncrementalSVD.#get_predicted_rati…
                 item_features      0 bytes  IncrementalSVD.#item_features
                   item_search      0 bytes  IncrementalSVD.#item_search
                         items      0 bytes  IncrementalSVD.#items
    load_book_crossing_dataset      0 bytes  IncrementalSVD.#load_book_crossing…
  load_large_movielens_dataset      0 bytes  IncrementalSVD.#load_large_moviele…
  load_small_movielens_dataset      0 bytes  IncrementalSVD.#load_small_moviele…
                          rmse      0 bytes  IncrementalSVD.#rmse
                 similar_items      0 bytes  IncrementalSVD.#similar_items
                 similar_us

In [18]:
rating_set = load_small_movielens_dataset();

Reusing existing downloaded files...


Loading ratings 100%|███████████████████████████████████| Time: 0:00:06


In [19]:
fieldnames(rating_set)

4-element Array{Symbol,1}:
 :training_set 
 :test_set     
 :user_to_index
 :item_to_index

In [20]:
# The format is (user, movie, mark)
rating_set.training_set

900188-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(4887, 211, 5.0) 
 IncrementalSVD.Rating(411, 2694, 4.0) 
 IncrementalSVD.Rating(5428, 504, 4.0) 
 IncrementalSVD.Rating(479, 559, 2.0)  
 IncrementalSVD.Rating(454, 468, 5.0)  
 IncrementalSVD.Rating(2018, 1128, 5.0)
 IncrementalSVD.Rating(400, 52, 3.0)   
 IncrementalSVD.Rating(3585, 2638, 3.0)
 IncrementalSVD.Rating(984, 771, 4.0)  
 IncrementalSVD.Rating(1220, 1827, 3.0)
 IncrementalSVD.Rating(165, 14, 3.0)   
 IncrementalSVD.Rating(5526, 2626, 4.0)
 IncrementalSVD.Rating(5182, 2956, 4.0)
 ⋮                                     
 IncrementalSVD.Rating(2038, 795, 4.0) 
 IncrementalSVD.Rating(5054, 154, 4.0) 
 IncrementalSVD.Rating(1272, 1076, 5.0)
 IncrementalSVD.Rating(1097, 536, 4.0) 
 IncrementalSVD.Rating(2676, 1606, 4.0)
 IncrementalSVD.Rating(463, 609, 3.0)  
 IncrementalSVD.Rating(5035, 529, 4.0) 
 IncrementalSVD.Rating(1579, 1086, 2.0)
 IncrementalSVD.Rating(2063, 2576, 3.0)
 IncrementalSVD.Rating(2245, 518,

In [21]:
rating_set.test_set

100021-element Array{IncrementalSVD.Rating,1}:
 IncrementalSVD.Rating(2589, 1107, 3.0)
 IncrementalSVD.Rating(933, 105, 3.0)  
 IncrementalSVD.Rating(4277, 110, 5.0) 
 IncrementalSVD.Rating(4345, 535, 4.0) 
 IncrementalSVD.Rating(5569, 640, 4.0) 
 IncrementalSVD.Rating(4169, 1136, 3.0)
 IncrementalSVD.Rating(1617, 877, 4.0) 
 IncrementalSVD.Rating(1246, 789, 5.0) 
 IncrementalSVD.Rating(5312, 1104, 3.0)
 IncrementalSVD.Rating(202, 397, 4.0)  
 IncrementalSVD.Rating(6036, 264, 4.0) 
 IncrementalSVD.Rating(3874, 865, 5.0) 
 IncrementalSVD.Rating(466, 291, 4.0)  
 ⋮                                     
 IncrementalSVD.Rating(5831, 749, 4.0) 
 IncrementalSVD.Rating(1968, 2389, 1.0)
 IncrementalSVD.Rating(2445, 433, 5.0) 
 IncrementalSVD.Rating(4258, 1848, 3.0)
 IncrementalSVD.Rating(3965, 292, 2.0) 
 IncrementalSVD.Rating(2773, 62, 4.0)  
 IncrementalSVD.Rating(5318, 433, 3.0) 
 IncrementalSVD.Rating(472, 289, 5.0)  
 IncrementalSVD.Rating(1780, 1241, 4.0)
 IncrementalSVD.Rating(2068, 1713

In [22]:
# Users and their IDs
rating_set.user_to_index

Dict{AbstractString,Int32} with 6040 entries:
  "4304" => 4304
  "3935" => 3935
  "5422" => 5422
  "5734" => 5734
  "2243" => 2243
  "1881" => 1881
  "5425" => 5425
  "4209" => 4209
  "1907" => 1907
  "2923" => 2923
  "599"  => 599
  "2491" => 2491
  "5944" => 5944
  "228"  => 228
  "2590" => 2590
  "3697" => 3697
  "5031" => 5031
  "2579" => 2579
  "5551" => 5551
  "1880" => 1880
  "2562" => 2562
  "3215" => 3215
  "3991" => 3991
  "4652" => 4652
  "4088" => 4088
  ⋮      => ⋮

In [23]:
# Movies and their IDs
rating_set.item_to_index

Dict{AbstractString,Int32} with 3706 entries:
  "Fried Green Tomatoes (1991)"                                       => 594
  "Milk Money (1994)"                                                 => 1361
  "From Russia with Love (1963)"                                      => 729
  "House II: The Second Story (1987)"                                 => 1247
  "Held Up (2000)"                                                    => 3549
  "Missing in Action 2: The Beginning (1985)"                         => 2177
  "Murder, My Sweet (1944)"                                           => 996
  "Hidden, The (1987)"                                                => 981
  "Cable Guy, The (1996)"                                             => 669
  "Big Kahuna, The (2000)"                                            => 893
  "Addams Family Values (1993)"                                       => 1857
  "Farinelli: il castrato (1994)"                                     => 1945
  "Education of Little T

In [24]:
# We can extract the titles ...
keys(rating_set.item_to_index)

Base.KeyIterator for a Dict{AbstractString,Int32} with 3706 entries. Keys:
  "Fried Green Tomatoes (1991)"
  "Milk Money (1994)"
  "From Russia with Love (1963)"
  "House II: The Second Story (1987)"
  "Held Up (2000)"
  "Missing in Action 2: The Beginning (1985)"
  "Murder, My Sweet (1944)"
  "Hidden, The (1987)"
  "Cable Guy, The (1996)"
  "Big Kahuna, The (2000)"
  "Addams Family Values (1993)"
  "Farinelli: il castrato (1994)"
  "Education of Little Tree, The (1997)"
  "In God's Hands (1998)"
  "Last Man Standing (1996)"
  "Sixth Sense, The (1999)"
  "Star Maps (1997)"
  "Girl, Interrupted (1999)"
  "Stand by Me (1986)"
  "Rob Roy (1995)"
  "Caligula (1980)"
  "Flirting With Disaster (1996)"
  "Hook (1991)"
  "Institute Benjamenta, or This Dream People Call Human Life (1995)"
  ⋮

In [25]:
# or codes
values(rating_set.item_to_index)

Base.ValueIterator for a Dict{AbstractString,Int32} with 3706 entries. Values:
  594
  1361
  729
  1247
  3549
  2177
  996
  981
  669
  893
  1857
  1945
  2759
  2814
  1840
  39
  2670
  32
  85
  449
  2532
  1290
  657
  3397
  ⋮

In [26]:
# Which movies did the user "3000" grade?
user_ratings(rating_set, "3000")

97-element Array{Tuple{Pair{Int32,SubString{String}},Float32},1}:
 (2177=>"Missing in Action 2: The Beginning (1985)", 5.0)     
 (3283=>"Jamaica Inn (1939)", 5.0)                            
 (802=>"Holy Man (1998)", 5.0)                                
 (2427=>"Morning After, The (1986)", 5.0)                     
 (2051=>"Bread and Chocolate (Pane e cioccolata) (1973)", 5.0)
 (772=>"Jerk, The (1979)", 5.0)                               
 (1527=>"Sliding Doors (1998)", 5.0)                          
 (854=>"Titan A.E. (2000)", 5.0)                              
 (3176=>"Goodbye, Lover (1999)", 5.0)                         
 (530=>"Sling Blade (1996)", 5.0)                             
 (2861=>"Tales of Terror (1962)", 5.0)                        
 (2611=>"Dersu Uzala (1974)", 5.0)                            
 (1524=>"Fantasia 2000 (1999)", 4.0)                          
 ⋮                                                            
 (2496=>"Poison Ivy: New Seduction (1997)", 2.0)    

In [27]:
# Let us find the exact title and code for "Blade Runner"
for k in keys(rating_set.item_to_index)
    if contains(k,"Blade")
        println(k)
    end
end

Sling Blade (1996)
Blade (1998)
Blade Runner (1982)
Some Folks Call It a Sling Blade (1993)


LoadError: [91mUnicodeError: invalid character index 12 (0xb3 is a continuation byte)[39m

In [28]:
get(rating_set.item_to_index,"Blade Runner (1982)",0)

744

In [29]:
# Did the user "3000" grade "Blade Runner" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Blade")
        println(k)
    end
end

LoadError: [91mMethodError: no method matching contains(::Pair{Int32,SubString{String}}, ::String)[0m
Closest candidates are:
  contains([91m::Function[39m, ::Any, [91m::Any[39m) at reduce.jl:664
  contains([91m::AbstractString[39m, ::AbstractString) at strings/search.jl:378[39m

In [30]:
# Did the user "3000" grade "Citizen Kane" ?
for k in user_ratings(rating_set,"3000")
    if contains(k[1],"Citizen")
        println(k)
    end
end

LoadError: [91mMethodError: no method matching contains(::Pair{Int32,SubString{String}}, ::String)[0m
Closest candidates are:
  contains([91m::Function[39m, ::Any, [91m::Any[39m) at reduce.jl:664
  contains([91m::AbstractString[39m, ::AbstractString) at strings/search.jl:378[39m

In [31]:
# This takes about 2.5 - 3.5 minutes
model = train(rating_set, 25);

Computing truncated rank 25 SVD 100%|███████████████████| Time: 0:03:30


In [32]:
fieldnames(model)

5-element Array{Symbol,1}:
 :user_to_index
 :item_to_index
 :U            
 :S            
 :V            

In [33]:
model.U

6040×25 Array{Float32,2}:
 0.0103889   0.0120029    0.0168709    …   0.0158769    0.01233   
 0.0144474   0.0114369    0.01046          0.0101324    0.0106126 
 0.0107866   0.00910588   0.0163534        0.00670826   0.02098   
 0.00651865  0.00644044   0.0152951        0.011859     0.0150761 
 0.0136352   0.0152549    0.00175968       0.00628066   0.0100611 
 0.0111944   0.017444     0.0156377    …   0.0115868    0.0159377 
 0.00798559  0.00724785   0.0180301        0.0185246    0.00868485
 0.0152403   0.0147618    0.0105188        0.0135337    0.0129277 
 0.0134772   0.0086194    0.0107375        0.0073613    0.00890968
 0.0188933   0.0127475   -0.0101799        0.0132478    0.0160951 
 0.0131323   0.0112366    0.0092004    …   0.00374553   0.00899902
 0.00637089  0.00853249   0.0146444        0.0127608    0.0173463 
 0.0125482   0.00996851   0.0126272        0.00664081   0.0157661 
 ⋮                                     ⋱                          
 0.00643649  0.0133206    0.013931  

In [34]:
model.S

25-element Array{Float32,1}:
 8190.01  
 2146.8   
  832.417 
 1146.95  
  902.182 
 1123.08  
  656.133 
  348.942 
  411.011 
  272.862 
  234.178 
  312.779 
  289.596 
  304.625 
  227.754 
  304.305 
  244.441 
  198.258 
  185.093 
  173.639 
  184.006 
  159.783 
  152.299 
  104.241 
   94.8967

In [35]:
model.V

3706×25 Array{Float32,2}:
 0.0358159   0.00761977  0.0678946   …  -0.0097144     0.00220286
 0.0250763   0.0153529   0.0197009       0.00250674    0.00897316
 0.0303364   0.0136331   0.0311693      -0.00813252    0.0169969 
 0.031914    0.0117044   0.0527089      -0.0119025     0.00203751
 0.0312843   0.00763712  0.0555989       0.0137012    -0.0107365 
 0.0345707   0.00455098  0.0691735   …  -0.0113717     0.00911906
 0.0305654   0.00853922  0.0291378       0.000823119   0.00253488
 0.0330744   0.00409456  0.0519298       0.000384519   0.0101533 
 0.0288623   0.00805691  0.0235771       0.00547914    0.0159415 
 0.0333413   0.0050427   0.0535728      -0.0126028     0.0312711 
 0.0300712   0.0053886   0.0386825   …   0.00967363   -0.00881841
 0.0168945   0.0282457   0.00881869      0.00195522    0.0164239 
 0.0257601   0.0194417   0.0134772       0.0104336     0.00799993
 ⋮                                   ⋱                           
 0.00119613  0.00259929  0.00406258      0.0146313

In [36]:
similar_items(model, "Friday the 13th (1980)")

10-element Array{SubString{String},1}:
 "Friday the 13th (1980)"                  
 "Amityville Horror, The (1979)"           
 "Jaws 2 (1978)"                           
 "Omen, The (1976)"                        
 "Friday the 13th Part 2 (1981)"           
 "Porky's (1981)"                          
 "Pet Sematary (1989)"                     
 "Cujo (1983)"                             
 "Battle for the Planet of the Apes (1973)"
 "Soylent Green (1973)"                    

In [39]:
# Take a look at the function
@which similar_items(model, "Friday the 13th (1980)")

similar_items(model::IncrementalSVD.RatingsModel, item) in IncrementalSVD at /home/slap/.julia/v0.6/IncrementalSVD/src/util.jl:113

In [40]:
similar_items(model, "Citizen Kane (1941)")

10-element Array{SubString{String},1}:
 "Citizen Kane (1941)"                                                        
 "M*A*S*H (1970)"                                                             
 "Lawrence of Arabia (1962)"                                                  
 "Chinatown (1974)"                                                           
 "Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)"
 "Rear Window (1954)"                                                         
 "Manchurian Candidate, The (1962)"                                           
 "Vertigo (1958)"                                                             
 "Casablanca (1942)"                                                          
 "Apocalypse Now (1979)"                                                      

In [41]:
similar_users(model,"3000")

10-element Array{SubString{String},1}:
 "3000"
 "2202"
 "5457"
 "1695"
 "5613"
 "3003"
 "5715"
 "3845"
 "1360"
 "4989"

In [44]:
# What is the opinion of user "3000" about "Blade Runner (1982)" 
# in the approximate model (no true mark) ?
get_predicted_rating(model, "3000", "Blade Runner (1982)")

4.1628504f0

In [45]:
# What is the opinion of user "3000" about "Citizen Kane (1941)"
# (no true mark!) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Citizen Kane (1941)")

4.0944686f0

In [47]:
# What is the opinion of user "3000" about "Holy Man (1998)"
# in the approximate model (true mark 5.0) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Holy Man (1998)")

1.682675f0

In [49]:
# What is the opinion of user "3000" about "Sling Blade (1996)"
# in the approximate model (true mark 5.0) ?
IncrementalSVD.get_predicted_rating(model, "3000", "Sling Blade (1996)")

4.089586f0

## Thank you for you attention

### Questions?